In [1]:
import pandas as pd
import numpy as np
import time
import os

In [ ]:
paises = ['Perú','Colombia','Ecuador']
#Pegar base sitios
sitios = pd.DataFrame()
for pais in paises:
    base = pd.read_excel('Noticias_Integración_'+pais+'_2022_blogs.xlsx')
    base['Country Code'] = base['Country Code'].fillna(0)
    base['Country Code'] = base['Country Code'].replace(0,base['Country Code'][0])
    sitios = sitios.append(base, ignore_index=True)
sitios.head(1)

In [ ]:
sitios['Mecanismo'] = sitios['Mecanismo'].fillna(0)
pd.unique(sitios['Mecanismo'])

In [ ]:
di = {'III':3,'II ': 2, 'II': 2, 'I':1,'I ':1}
sitios['Mecanismo'] = sitios['Mecanismo'].map(di).fillna(0)
pd.unique(sitios['Mecanismo'])

In [ ]:
sitios['Mecanismo A'] = sitios['Mecanismo A'].fillna(0)
print(pd.unique(sitios['Mecanismo A']))
di = {'III':3,'II ': 2, 'II': 2, 'I':1,'I ':1,'<<aqdrt eb,':0,'i':1,'ii':2}
sitios['Mecanismo A'] = sitios['Mecanismo A'].map(di).fillna(0)
pd.unique(sitios['Mecanismo A'])

In [ ]:
sitios['Mecanismo B'] = sitios['Mecanismo B'].fillna(0)
print(pd.unique(sitios['Mecanismo B']))
di = {'III':3,'II ': 2, 'II': 2, 'I':1,'I ':1,'<<aqdrt eb,':0,'i':1,'ii':2}
sitios['Mecanismo B'] = sitios['Mecanismo B'].map(di).fillna(0)
pd.unique(sitios['Mecanismo B'])

In [ ]:
sitios['Mecanismo'] = np.where(sitios['Country Code'] == 'COL', sitios['Mecanismo A'], sitios['Mecanismo'])
sitios.head(1)

In [ ]:
#Explorar formato datetime
index_date = pd.to_datetime(sitios['Date'][0], format='%Y/%m/%d', 
                                    exact = False)
index_day_of_week = index_date.day_name()
print(index_day_of_week)
print(sitios['Date'][0])
sitios.set_index('Date', inplace=True)
sitios.head(2)

In [ ]:
sitios['Mecanismo 1'] = np.where(sitios['Mecanismo'] == 1, 1, 0)
sitios['Mecanismo 2'] = np.where(sitios['Mecanismo'] == 2, 1, 0)
sitios['Mecanismo 3'] = np.where(sitios['Mecanismo'] == 3, 1, 0)

In [3]:
def new_case_count(state_new_cases, numero_mecanismo = None):
    i = 1
    cent = list()
    if numero_mecanismo == None: numero_mecanismo = 'Mecanismo'
    returned_data = pd.DataFrame()
    first_Monday_found = False
    week_case_count = 0
    week_case_counts = []
    for index, value in state_new_cases.iterrows():
        try:
            index_date = pd.to_datetime(index, format='%Y/%m/%d', 
                                        exact = False)
        except:
            index_date = pd.to_datetime(np.nan, format='%Y/%m/%d', 
                                        exact = False)
        index_day_of_week = index_date.day_name()
        if not first_Monday_found and index_day_of_week != 'Monday':
            continue
        first_Monday_found = True
        week_case_count += value[numero_mecanismo]
        if index_day_of_week == 'Sunday':
            week_case_counts.append(week_case_count)
            df2 = pd.DataFrame({numero_mecanismo: week_case_count}, index=[index_date.date()])
            if i == 1:
                returned_data = pd.concat([returned_data, df2])
                i += 1
            else:
                if index_date.date() in cent:
                    i += 1
                else:
                    returned_data = pd.concat([returned_data, df2])
            cent.append(index_date.date())         
            week_case_count = 0
            di = dict()
    return returned_data

In [19]:
def pegado_bases(df, paises):
    for pais in paises:
        new_cases_weekly_total_method_1 = pd.DataFrame(new_case_count(df[df['Country Code'] == pais],'Mecanismo 1'))
        new_cases_weekly_total_method_2 = pd.DataFrame(new_case_count(df[df['Country Code'] == pais],'Mecanismo 2'))
        merged = pd.merge(new_cases_weekly_total_method_1,new_cases_weekly_total_method_2, left_on=new_cases_weekly_total_method_1.index, right_on=new_cases_weekly_total_method_2.index)
        merged = merged.rename(columns={"key_0": "Fecha"})
        new_cases_weekly_total_method_3 = pd.DataFrame(new_case_count(df[df['Country Code'] == pais],'Mecanismo 3'))
        merged = pd.merge(merged,new_cases_weekly_total_method_3, left_on=merged.Fecha, right_on=new_cases_weekly_total_method_3.index)
        merged = merged.drop('key_0', axis = 'columns')
        merged.set_index('Fecha', inplace=True)
        merged['Pais'] = pais
        if pais == 'COL':
            COL = merged
        elif pais == 'PER':
            print("ENTRÉ")
            PER = merged
            print(PER.head(2))
        else:
            ECU = merged
    base_1 = COL.append(PER)
    base_final = base_1.append(ECU)
    print(base_final.head(2))
    return base_final

In [20]:
pegado_bases(sitios, ['COL','PER','ECU']).to_excel('Noticias_Integración_2022_blogs_desagregado_semanal.xlsx')

NameError: name 'sitios' is not defined

In [ ]:
sitios_limpio = pd.DataFrame({'Pais':['Colombia','Colombia','Colombia',
                                      'Ecuador','Ecuador','Ecuador',
                                      'Peru','Peru','Peru'],
                              'Variable':['Mecanismo I','Mecanismo II','Mecanismo III',
                                      'Mecanismo I','Mecanismo II','Mecanismo III',
                                      'Mecanismo I','Mecanismo II','Mecanismo III'],
                              'Fecha':['28/02/2022','28/02/2022','28/02/2022','28/02/2022','28/02/2022','28/02/2022','28/02/2022','28/02/2022','28/02/2022'],
                              'Dato':[212, 79,176,
                                     1,5,10,
                                     14,46,163]})
sitios_limpio.to_excel('Noticias_Integración_2022_blogs.xlsx')

In [21]:
paises_twitter = ['Perú','Colombia','Ecuador']
#Pegar base Twitter
twitter = pd.DataFrame()
for pais in paises_twitter:
    base = pd.read_excel('Noticias_Integración_'+pais+'_2022_Twitter.xlsx')
    base['Country Code'] = base['Country Code'].fillna(0)
    base['Country Code'] = base['Country Code'].replace(0,base['Country Code'][0])
    twitter = twitter.append(base, ignore_index=True)
print(pd.unique(twitter['Mecanismo']))
twitter['Mecanismo'] = twitter['Mecanismo'].fillna(0)
print(pd.unique(twitter['Mecanismo']))
di = {'III':3,'II ': 2, 'II': 2, 'I':1,'I ':1,'II Y III':3, 'III ':3}
twitter['Mecanismo'] = twitter['Mecanismo'].map(di).fillna(0)
print(pd.unique(twitter['Mecanismo']))
pd.unique(twitter['Country Code'])
twitter['Mecanismo 1'] = np.where(twitter['Mecanismo'] == 1, 1, 0)
twitter['Mecanismo 2'] = np.where(twitter['Mecanismo'] == 2, 1, 0)
twitter['Mecanismo 3'] = np.where(twitter['Mecanismo'] == 3, 1, 0)
twitter.set_index('Date', inplace=True)
print(twitter.head(1))

['III' nan 'II ' 'II' 'I' 'III ' 'I ' 'II Y III']
['III' 0 'II ' 'II' 'I' 'III ' 'I ' 'II Y III']
[3. 0. 2. 1.]
                                                                     Title  \
Date                                                                         
2022-08-14 12:56:08.000  ¿Populismo? Expulsar extranjeros no soluciona ...   

                         Mecanismo  \
Date                                 
2022-08-14 12:56:08.000        3.0   

                                                                   Snippet  \
Date                                                                         
2022-08-14 12:56:08.000  ¿Populismo? Expulsar extranjeros no soluciona ...   

                                                                       Url  \
Date                                                                         
2022-08-14 12:56:08.000  http://twitter.com/larepublica_pe/statuses/155...   

                              Domain Sentiment Country Code Continen

In [22]:
pegado_bases(twitter, ['PER','COL','ECU']).to_excel('Noticias_Integración_2022_twitter_desagregado_semanal.xlsx')

ENTRÉ
Empty DataFrame
Columns: [Pais]
Index: []
            Mecanismo 1  Mecanismo 2  Mecanismo 3 Pais
Fecha                                                 
2022-12-25          0.0          0.0          0.0  COL
2022-12-18          1.0          0.0          3.0  COL


In [16]:
new_case_count(twitter[twitter['Country Code'] == 'PER'], numero_mecanismo = 'Mecanismo 1')

""


In [ ]:
new_case_count(twitter[twitter['Country Code'] == 'PER'], numero_mecanismo = 'Mecanismo 2')

In [ ]:
new_case_count(twitter[twitter['Country Code'] == 'PER'], numero_mecanismo = 'Mecanismo 3')

In [ ]:
new_case_count(twitter[twitter['Country Code'] == 'PER'])

In [ ]:

twitter_limpio = pd.DataFrame({'Pais':['Colombia','Colombia','Colombia',
                                      'Ecuador','Ecuador','Ecuador',
                                      'Peru','Peru','Peru'],
                              'Variable':['Mecanismo I','Mecanismo II','Mecanismo III',
                                      'Mecanismo I','Mecanismo II','Mecanismo III',
                                      'Mecanismo I','Mecanismo II','Mecanismo III'],
                              'Fecha':['28/02/2022','28/02/2022','28/02/2022','28/02/2022','28/02/2022','28/02/2022','28/02/2022','28/02/2022','28/02/2022'],
                              'Dato':[74, 0,1,
                                     110,3,5,
                                     319,7,12]})
twitter_limpio.to_excel('Noticias_Integración_2022_twitter.xlsx')